# Commercial data access through Sentinel Hub

This notebook can be used to access commercial high resolution satellite data that have been ordered through Sentinel Hub.

To order commercial data, use the [Commercial_data_SentinelHub_order notebook](Commercial_data_SentinelHub_order.ipynb).

Data from the following data providers or missions can be ordered:

* AIRBUS [Pleiades](https://docs.sentinel-hub.com/api/latest/data/airbus/pleiades/) & [SPOT](https://docs.sentinel-hub.com/api/latest/data/airbus/spot/)
* [Planet SCOPE](https://docs.sentinel-hub.com/api/latest/data/planet/planet-scope/)
* [Planet SkySat](https://docs.sentinel-hub.com/api/latest/data/planet/skysat/)
* [WorldView](https://docs.sentinel-hub.com/api/latest/data/maxar/world-view/)

A [Sentinel Hub](https://www.sentinel-hub.com/) account with sufficient credit is required. You'll need to provide your `client id` and `client secret` or set the relevant environment variables.

This example will use AIRBUS SPOT data.

## Load Python packages and configure Sentinel Hub connection

In [1]:
%matplotlib inline

import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import datetime
from sentinelhub import DataCollection, ResamplingType, SHConfig, SentinelHubBYOC, BBox, CRS
import pandas

from datacube.utils.cog import write_cog
from datacube.utils.geometry import CRS, assign_crs

from odc_sh import engine, SentinelHubCommercialData

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


Set Sentinel HUB credentials

In [2]:
sh_client_id=""
sh_client_secret=""

if not sh_client_id:
    sh_client_id = os.environ['SH_CLIENT_ID']

if not sh_client_secret:
    sh_client_secret = os.environ['SH_CLIENT_SECRET']

In [3]:
config = SHConfig()
config.sh_client_id = sh_client_id
config.sh_client_secret = sh_client_secret

shcd = SentinelHubCommercialData(config)

## Define data search parameters

Following parameters are required to query and load data:

* `latitude`: min and max latitude
* `longitude`: min and max longitude
* `time`: start and end date and time
* `resolution`: output resolution in meters
* `collection_id`: unique id of the data collection

In [4]:
resolution = 1.5  # in meters
longitude = (36.83, 36.90)
latitude = (-17.7, -17.6)
time = ("2021-01-01", "2021-12-30")

Find all compatible collections

In [7]:
query = {'provider': 'AIRBUS',
         'bounds': {'bbox': [longitude[0], latitude[0], longitude[1], latitude[1]]},
         'data': [{'constellation': 'SPOT'}]}

collections = shcd.get_collection(query)
collections.print_info()

    idx                                   id                       name                           created
--  ------------------------------------  -----------------------  ---------------------------  ---------
 0  4dc28550-8f1e-4e50-a849-a3681c9cbc60  My Airbus Spot data      2022-11-30T10:06:10.673456Z          0
 1  77c15005-007e-41f3-9644-b0143bdaf472  My Airbus Pleiades data  2022-11-30T10:05:59.686626Z          0
 2  8aa730c0-064d-4738-89ea-276073543bab  Mozambique               2023-03-03T04:34:30.502481Z          0
 3  f17ee64c-2533-4958-b2f8-1681e0e17749  Rwanda                   2023-03-03T04:30:53.784682Z          0


Select the desired collection id

In [8]:
collection_id = "8aa730c0-064d-4738-89ea-276073543bab"

## Connect to Datacube and load data

In [9]:
dc = engine.Datacube(sh_client_id=sh_client_id, sh_client_secret=sh_client_secret)

In [10]:
myCollection = dc.get_BYOC_collection(collection_id)
myCollection

<DataCollection.8aa730c0-064d-4738-89ea-276073543bab: DataCollectionDefinition(
  api_id: byoc-8aa730c0-064d-4738-89ea-276073543bab
  catalog_id: byoc-8aa730c0-064d-4738-89ea-276073543bab
  wfs_id: byoc-8aa730c0-064d-4738-89ea-276073543bab
  service_url: https://services.sentinel-hub.com
  collection_type: BYOC
  bands: (Band(name='B0', units=(<Unit.DN: 'DN'>,), output_types=(<class 'numpy.uint16'>,)), Band(name='B1', units=(<Unit.DN: 'DN'>,), output_types=(<class 'numpy.uint16'>,)), Band(name='B2', units=(<Unit.DN: 'DN'>,), output_types=(<class 'numpy.uint16'>,)), Band(name='B3', units=(<Unit.DN: 'DN'>,), output_types=(<class 'numpy.uint16'>,)), Band(name='PAN', units=(<Unit.DN: 'DN'>,), output_types=(<class 'numpy.uint16'>,)))
  collection_id: 8aa730c0-064d-4738-89ea-276073543bab
  is_timeless: False
  has_cloud_coverage: False
)>

Metadata for a data collection includes available bands or measurements. When loading data, by default all bands will be retrieved. If a list of band names are provided through the `measurements` parameter, only the selected bands will be retrieved.

If the output resolution and grid doesn't match the input data, nearest neighbor resampling is applied by default. An alternative resampling method can be configured by setting the `sh_resampling` parameter. Options include `ResamplingType.BICUBIC`, `ResamplingType.BILINEAR` and `ResamplingType.NEAREST`

In [11]:
ds = dc.load(
    product=myCollection,
    latitude=latitude,
    longitude=longitude,
    time=time,
    sh_resolution=resolution,
    #sh_resampling=ResamplingType.BILINEAR
    #measurements=["PAN", "B2", "B1","B0"],
)

Searching for new products
measurement: {'name': 'B0', 'units': 'DN', 'dtype': 'uint16', 'nodata': 0}
measurement: {'name': 'B1', 'units': 'DN', 'dtype': 'uint16', 'nodata': 0}
measurement: {'name': 'B2', 'units': 'DN', 'dtype': 'uint16', 'nodata': 0}
measurement: {'name': 'B3', 'units': 'DN', 'dtype': 'uint16', 'nodata': 0}
measurement: {'name': 'PAN', 'units': 'DN', 'dtype': 'uint16', 'nodata': 0}
Product created for 8aa730c0-064d-4738-89ea-276073543bab
LOADING SENTINEL HUB DATA
---------------------------------------------
longitude: 36.83, 36.9; latitude: -17.7, -17.6; resolution: 1.5 m; crs: EPSG:4326; time: 2021-02-26; resampling: ResamplingType.BICUBIC, max cloud coverage: None


  0%|          | 0/12 [00:00<?, ?it/s]

---------------------------------------------
longitude: 36.83, 36.9; latitude: -17.7, -17.6; resolution: 1.5 m; crs: EPSG:4326; time: 2021-06-29; resampling: ResamplingType.BICUBIC, max cloud coverage: None


  0%|          | 0/12 [00:00<?, ?it/s]

## Pan-sharpening

Even though data of all spectral bands are loaded with the same spatial resolution, they may have different native resolutions. 

For SPOT data, we can use the panchromatic band (1.5 meter resolution) to sharpen the Red, Green, Blue band images.

In [ ]:
ds = ds.to_dataset(dim="bands")
weight = (ds.B2 + ds.B1 + ds.B0* 0.4) / 2.4;
ratio = ds.PAN/weight #* 2.5
sharpened = ds[['B2', 'B1', 'B0']]/ 10000 * ratio

In [ ]:
# Visualize the sharpened images
sharpened[['B2', 'B1', 'B0']].to_array().plot.imshow(col='time', robust=True)

## Save data as Cloud-Optimzed Geotiff (COGs)

Data can be analysed directly or saved for use in a different workflow. Below, we will save data from each date as a multi-band COG.

We will add the CRS to the dataset attribute, so it can saved using the `write_cog` function.

In [ ]:
sharpened = assign_crs(sharpened, crs='EPSG:4326')

In [ ]:
for idx in range(len(ds.time)):
    
    # Select a single time-slice
    rgb_tiff = sharpened[['B2', 'B1', 'B0']].isel(time=idx).to_array()

    assign_crs(rgb_tiff, crs='EPSG:4326')
    # Write multi-band GeoTIFF to a location
    write_cog(rgb_tiff,
          fname=f'{country}_{str(ds.time.values[idx])[:10]}.tif',
          overwrite=True)

### Save as RGB image

A RGB image with 3 channels and values ranging from 0 to 255 can be visualized easily and is used as input in some image analysis workflows.

We will rescale the reflectance values of 0 to 0.3 to 0 to 255 and assign 255 to reflectance of greater than 0.3.

In [ ]:
rgb = (sharpened.where(sharpened<=0.3,0.3)*255/0.3).round().astype(np.uint8)

In [ ]:
for idx in range(len(ds.time)):
    
    # Select a single time-slice
    rgb_tiff = rgb[['B2', 'B1', 'B0']].isel(time=idx).to_array()

    assign_crs(rgb_tiff, crs='EPSG:4326')
    # Write multi-band GeoTIFF to a location
    write_cog(rgb_tiff,
          fname=f'{country}_{str(ds.time.values[idx])[:10]}_rgb.tif',
          overwrite=True)
